# Lambda phage

In [39]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.notebooks.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

plt.style.use("./scripts/notebooks/custom_style.mplstyle")
%matplotlib inline

In [40]:
slice_vec = np.zeros(5)

## Load initial data

In [41]:
with np.load("scripts/reference_solutions/lp_ode_ref_r5.npz") as data:
    P_exact = data["P_full"]
    P_best_approximation_r5 = data["P_best_approximation"]
    walltime_exact = data["wall_time"]

In [42]:
with np.load("scripts/reference_solutions/lambda_phage_ssa_1e+04.npz") as data:
    ssa_1e4 = data["result"]
    walltime_ssa_1e4 = data["wall_time"]

ssa_1e4_sol = SSASol(ssa_1e4)
P_ssa_1e4 = ssa_1e4_sol.calculateFullDistribution()

In [43]:
with np.load("scripts/reference_solutions/lambda_phage_ssa_1e+05.npz") as data:
    ssa_1e5 = data["result"]
    walltime_ssa_1e5 = data["wall_time"]

ssa_1e5_sol = SSASol(ssa_1e5)
P_ssa_1e5 = ssa_1e5_sol.calculateFullDistribution()

In [44]:
with np.load("scripts/reference_solutions/lambda_phage_ssa_1e+06.npz") as data:
    ssa_1e6 = data["result"]
    walltime_ssa_1e6 = data["wall_time"]

ssa_1e6_sol = SSASol(ssa_1e6)
P_ssa_1e6 = ssa_1e6_sol.calculateFullDistribution()

## Comparison of TTN and SSA results with matrix integrator

### Load data

#### Get walltimes

In [45]:
time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau1e-3")
walltime_p0_r_3_3 = time_series.getWallTime()

In [46]:
time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
walltime_p0_r_5_4 = time_series.getWallTime()

In [47]:
time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
walltime_p0_r_5_5 = time_series.getWallTime()

In [48]:
time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-3")
walltime_p0_r_6_6 = time_series.getWallTime()

In [49]:
time_series = TimeSeries("output/lambda_phage_pfull_r5_e_tau1e-3")
walltime_pfull = time_series.getWallTime()

In [50]:
walltimes = [walltime_exact, walltime_ssa_1e6, walltime_ssa_1e5, walltime_ssa_1e4, walltime_p0_r_6_6, walltime_p0_r_5_5, walltime_p0_r_5_4, walltime_p0_r_3_3]
labels = ["exact", "$10^6$ runs", "$10^5$ runs", "$10^4$ runs", "$r = (6, 6)$", "$r = (5, 5)$", "$r = (5, 4)$", "$r = (3, 3)$"]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
color = ["grey", colors[1], colors[1], colors[1], colors[0], colors[0], colors[0], colors[0]]
bar_labels = ['_exact', 'SSA', '_SSA', '_SSA', 'PS-TTN', '_PS-TTN', '_PS-TTN', '_PS-TTN']

### Plots

#### Error between TTN integrator and the matrix integrator reference solution

In [51]:
time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau1e-3")
time = time_series.time
P_p0_r3_3_tau_1e3 = time_series.calculateFullDistribution()
err_r_3_3 = np.linalg.norm(P_p0_r3_3_tau_1e3 - P_exact, axis=1)
rel_err_r_3_3 = err_r_3_3 / np.linalg.norm(P_exact, axis=1)

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
P_p0_r5_4_tau_1e3 = time_series.calculateFullDistribution()
err_r_5_4 = np.linalg.norm(P_p0_r5_4_tau_1e3 - P_exact, axis=1)
rel_err_r_5_4 = err_r_5_4 / np.linalg.norm(P_exact, axis=1)

time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
P_p0_r5_5_tau_1e3 = time_series.calculateFullDistribution()
err_r_5_5 = np.linalg.norm(P_p0_r5_5_tau_1e3 - P_exact, axis=1)
rel_err_r_5_5 = err_r_5_5 / np.linalg.norm(P_exact, axis=1)

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-3")
P_p0_r6_6_tau_1e3 = time_series.calculateFullDistribution()
err_r_6_6 = np.linalg.norm(P_p0_r6_6_tau_1e3 - P_exact, axis=1)
rel_err_r_6_6 = err_r_6_6 / np.linalg.norm(P_exact, axis=1)

In [52]:
def calculateSSAError(P_ssa, P_exact, ssa_sol):
    grid_liml = np.array([0, 0, 0, 0 ,0], dtype="int64")
    grid_n = np.array([16, 41, 11, 11, 11], dtype="int64")

    n_min = np.amax(np.stack((grid_liml, ssa_sol.n_min)), axis=0)
    n_max = np.amin(np.stack((grid_n + grid_liml, ssa_sol.n_max)), axis=0)
    n = n_max - n_min
    dx = np.prod(n)
    t = P_ssa.shape[0]
    P_ssa_red = np.zeros((t, dx))
    P_exact_red = np.zeros((t, dx))

    vec_index = np.zeros(n.size, dtype="int64")
    for i in range(P_exact.shape[1]):
        if np.all(vec_index < n):
            new_index = vecIndexToCombIndex(vec_index, n)
            P_exact_red[:, new_index] = P_exact[:, i]
        incrVecIndex(vec_index, grid_n, grid_n.size)

    vec_index = np.zeros(n.size, dtype="int64")
    for i in range(P_ssa.shape[1]):
        if np.all(vec_index < n):
            new_index = vecIndexToCombIndex(vec_index, n)
            P_ssa_red[:, new_index] = P_ssa[:, i]
        incrVecIndex(vec_index, ssa_sol.n, ssa_sol.n.size)

    return np.linalg.norm(P_ssa_red - P_exact_red, axis=1)

In [53]:
err_ssa_1e4 = calculateSSAError(P_ssa_1e4, P_exact, ssa_1e4_sol)
err_ssa_1e5 = calculateSSAError(P_ssa_1e5, P_exact, ssa_1e5_sol)
err_ssa_1e6 = calculateSSAError(P_ssa_1e6, P_exact, ssa_1e6_sol)

In [54]:
err_best_approximation_r5 = np.linalg.norm(P_best_approximation_r5 - P_exact, axis=1)

### Plots

In [ ]:
gs = gridspec.GridSpec(1, 30)
fig = plt.figure(figsize=(8.5, 3))

general_labels = ["best-approximation ($r=5$)"]

ax0 = plt.subplot(gs[0, :9])
ax0.plot(time[1:], err_r_3_3[1:], '.-', label="$r = (3, 3)$")
ax0.plot(time[1:], err_r_5_4[1:], '.-', label="$r = (5, 4)$")
ax0.plot(time[1:], err_r_5_5[1:], '.-', label="$r = (5, 5)$")
ax0.plot(time[1:], err_r_6_6[1:], '.-', label="$r = (6, 6)$")
l1, = ax0.plot(time[1:], err_best_approximation_r5[1:], 'k:')

ax0.set_yscale("log")
ax0.set_xlabel("$t$")
ax0.legend(ncols=1)
ax0.set_title("$\Vert P^\mathrm{{TTN}}(t,\mathbf{{x}})-P^\mathrm{{ref}}(t,\mathbf{{x}}) \Vert$")

ax1 = plt.subplot(gs[0, 9:18])
ax1.plot(time[1:], err_ssa_1e4[1:], 'x-', label="$10^4$ runs")
ax1.plot(time[1:], err_ssa_1e5[1:], 'x-', label="$10^5$ runs")
ax1.plot(time[1:], err_ssa_1e6[1:], 'x-', label="$10^6$ runs")
ax1.plot(time[1:], err_best_approximation_r5[1:], 'k:')

ax1.set_yscale("log")
ax1.set_xlabel("$t$")
ax1.legend(ncols=1)
ax1.set_title("$\Vert P^\mathrm{{SSA}}(t,\mathbf{{x}})-P^\mathrm{{ref}}(t,\mathbf{{x}}) \Vert$")
ax1.set_yticklabels([])

plt.setp((ax0, ax1), ylim=[1e-4, 2e-1], xticks=[1, 4, 7, 10])

ax2 = plt.subplot(gs[0:, 22:])
ax2.barh(labels, walltimes, label=bar_labels, color=color)
ax2.set_xscale("log")
ax2.set_xlabel("wall time [$\mathrm{{s}}$]")
ax2.legend()

plt.subplots_adjust(wspace=0.3)
plt.savefig("plots/lambda_phage_comparison_ttn_ssa.pdf", bbox_inches="tight");

## Error depending on time step size

In [56]:
dm_max_r_6_6 = []
dm_max_r_5_5 = []
dm_max_r_5_4 = []
dm_max_r_3_3 = []
dm_max_r_6_6_i = []

err_r_6_6 = []
err_r_5_5 = []
err_r_5_4 = []
err_r_3_3 = []
err_r_6_6_i = []

tau = []

time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau5e-1")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau5e-1")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau5e-1")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau5e-1")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau5e-1")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau2e-1")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau2e-1")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau2e-1")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau2e-1")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau2e-1")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-1")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-1")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau1e-1")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-1")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau1e-1")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau5e-2")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau5e-2")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau5e-2")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau5e-2")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau5e-2")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau2e-2")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau2e-2")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau2e-2")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau2e-2")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau2e-2")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-2")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-2")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau1e-2")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-2")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau1e-2")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau5e-3")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau5e-3")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau5e-3")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau5e-3")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau5e-3")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau2e-3")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau2e-3")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau2e-3")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau2e-3")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau2e-3")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())


time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_5.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
dm_max_r_5_4.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_5_4.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r3-3_e_tau1e-3")
dm_max_r_3_3.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_3_3.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-3")
dm_max_r_6_6.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))

time_series = TimeSeries("output/lambda_phage_p0_r6-6_i_tau1e-3")
dm_max_r_6_6_i.append(np.abs(time_series.getMaxMassErr()))
P_ttn = time_series.calculateFullDistribution()
err_r_6_6_i.append(np.linalg.norm(P_ttn[-1] - P_exact[-1], ord=None))
tau.append(time_series.getTau())

### Plot

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3), sharex="row")
t = np.linspace(0.005, 0.5)

general_labels = ["$r=(3, 3)$", "$r=(5, 4)$", "$r=(5, 5)$", "$r=(6, 6)$"]

l1, = ax1.loglog(tau, err_r_3_3, '.-')
l2, = ax1.loglog(tau, err_r_5_4, '.-')
l3, = ax1.loglog(tau, err_r_5_5, '.-')
l4, = ax1.loglog(tau, err_r_6_6, '.-')
ax1.loglog(tau, err_r_6_6_i, '-.', color=plt.rcParams['axes.prop_cycle'].by_key()['color'][3])
ax1.loglog(t, t**1*0.02, 'k:', label="$\Delta t$")
ax1.set_xlabel("$\Delta t$")
ax1.set_ylabel("$\Vert P^\mathrm{{TTN}}(10,\mathbf{{x}})-P^\mathrm{{ref}}(10,\mathbf{{x}}) \Vert$")
ax1.set_ylim([5e-5, 5e1])
ax1.legend()

ax2.loglog(tau, dm_max_r_3_3, '.-')
ax2.loglog(tau, dm_max_r_5_4, '.-')
ax2.loglog(tau, dm_max_r_5_5, '.-')
ax2.loglog(tau, dm_max_r_6_6, '.-')
ax2.loglog(tau, dm_max_r_6_6_i, '-.', color=plt.rcParams['axes.prop_cycle'].by_key()['color'][3])
ax2.loglog(t, t**2*0.05, 'k--', label="$(\Delta t)^2$")
ax2.set_xlabel("$\Delta t$")
ax2.set_ylabel("$\max_t(|\Delta m(t)|)$")
ax2.set_ylim([5e-7, 5e1])
ax2.legend()

fig.legend([l1, l2, l3, l4], general_labels, loc="upper center", ncol=5)
plt.subplots_adjust(wspace=0.4)
plt.savefig("plots/lambda_phage_time_err_ttn.pdf", bbox_inches="tight");

## Comparison with deterministic solution

In [58]:
time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
concentrations = time_series.calculateMoments()
t = time_series.time

In [59]:
from scripts.models.lambda_phage_gillespy import model
concentrations_ode = model.run(algorithm="ODE")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 4))
deviation = {key: np.sqrt(concentrations[1][key] - concentrations[0][key]**2) for key in concentrations[0]}
observables = ["S1", "S2"]
observables_alt = ["$S_0$", "$S_1$", "$S_2$"]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    ax1.plot(t, concentrations[0][o], '-', label=o_alt, color=colors[i], alpha=0.7)
    ax1.fill_between(t, concentrations[0][o]-deviation[o], concentrations[0][o]+deviation[o], color=colors[i], alpha=.2)
    ax1.plot(concentrations_ode.data[0]["time"], concentrations_ode.data[0][o], '--', color=colors[i], alpha=1.0)
ax1.set_ylabel("$\langle x_i(t) \\rangle$")
ax1.set_ylim([0.0, 20.0])

observables = ["S3", "S4", "S5"]
observables_alt = ["$S_2$", "$S_3$", "$S_4$"]
for idx_o, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    i = idx_o + 2
    ax2.plot(t, concentrations[0][o], '-', label=o_alt, color=colors[i], alpha=0.7)
    ax2.fill_between(t, concentrations[0][o]-deviation[o], concentrations[0][o]+deviation[o], color=colors[i], alpha=.2)
    ax2.plot(concentrations_ode.data[0]["time"], concentrations_ode.data[0][o], '--', color=colors[i], alpha=1.0)
ax2.set_ylabel("$\langle x_i(t) \\rangle$")
ax2.set_ylim([0, 2.5])
ax2.yaxis.tick_right()
ax2.yaxis.set_ticks_position("both")
ax2.yaxis.set_label_position("right")
plt.setp((ax1, ax2), xlabel="$t$", xlim=[0.0, 10.0], xticks=[0.0, 2.5, 5.0, 7.5, 10.0]);

lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(ll, []) for ll in zip(*lines_labels)]
fig.legend(lines, labels, ncols=5, loc="upper center")

plt.savefig("plots/lambda_phage_concentrations.pdf");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), layout='constrained')
observables = ["S1", "S2", "S3", "S4", "S5"]
for i, o in enumerate(observables):
    ax.plot(t, np.abs((concentrations[0][o]-concentrations_ode.data[0][o])), '-', label="$S_{{{}}}$".format(i))
ax.set_xlabel("$t$")
ax.set_ylabel("$|c_i^\mathrm{{CME}}(t) - c_i^\mathrm{{ODE}}(t)|$");
fig.legend(*ax.get_legend_handles_labels(), loc="outside center right")

plt.savefig("plots/lambda_phage_concentrations_err.pdf");

## Error depending on rank and partition

### Load data

#### Partition 0

In [62]:
time_series = TimeSeries("output/lambda_phage_p0_r5-3_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_00 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_10 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_20 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_30 = np.linalg.norm(P - P_exact, axis=1, ord=None)

#### Partition 1

In [63]:
time_series = TimeSeries("output/lambda_phage_p1_r5-3_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_01 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p1_r5-4_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_11 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p1_r5-5_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_21 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p1_r6-6_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_31 = np.linalg.norm(P - P_exact, axis=1, ord=None)

#### Partition 2

In [64]:
time_series = TimeSeries("output/lambda_phage_p2_r5-3_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_02 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p2_r5-4_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_12 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p2_r5-5_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_22 = np.linalg.norm(P - P_exact, axis=1, ord=None)

time_series = TimeSeries("output/lambda_phage_p2_r6-6_e_tau1e-3")
P = time_series.calculateFullDistribution()
err_32 = np.linalg.norm(P - P_exact, axis=1, ord=None)

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(6, 5))

axs[0, 0].plot(time[1:], err_00[1:], '.-', label="$\mathcal{{P}}_0$")
axs[0, 0].plot(time[1:], err_01[1:], '.-', label="$\mathcal{{P}}_1$")
axs[0, 0].plot(time[1:], err_02[1:], '.-', label="$\mathcal{{P}}_2$")
axs[0, 0].set_title("$r = (5, 3)$")

axs[0, 1].plot(time[1:], err_10[1:], '.-')
axs[0, 1].plot(time[1:], err_11[1:], '.-')
axs[0, 1].plot(time[1:], err_12[1:], '.-')
axs[0, 1].set_title("$r = (5, 4)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

axs[1, 0].plot(time[1:], err_20[1:], '.-')
axs[1, 0].plot(time[1:], err_21[1:], '.-')
axs[1, 0].plot(time[1:], err_22[1:], '.-')
axs[1, 0].set_title("$r = (5, 5)$")

axs[1, 1].plot(time[1:], err_30[1:], '.-')
axs[1, 1].plot(time[1:], err_31[1:], '.-')
axs[1, 1].plot(time[1:], err_32[1:], '.-')
axs[1, 1].set_title("$r = (6, 6)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs, xlabel="$t$", xticks=[1.0, 4.0, 7.0, 10.0], ylim=[3e-4, 2e-2], yscale="log")
plt.subplots_adjust(hspace=0.4)
fig.suptitle("$\Vert P^\mathrm{{TTN}}(t,\mathbf{{x}})-P^\mathrm{{ref}}(t,\mathbf{{x}}) \Vert$", fontsize=14, y=1.03)
fig.legend(*axs[0, 0].get_legend_handles_labels(), ncols=3, loc="center", bbox_to_anchor=(0.5, 0.93))
plt.tight_layout()
plt.savefig("plots/lambda_phage_err_comparison_ttn_alt.pdf", bbox_inches="tight");

## Mass error

### Load data

#### Partition 0

In [66]:
time_series = TimeSeries("output/lambda_phage_p0_r5-3_e_tau1e-3")
mass_err00 = np.abs(time_series.getMassErr())
time = time_series.time

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
mass_err10 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
mass_err20 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-3")
mass_err30 = np.abs(time_series.getMassErr())

#### Partition 1

In [67]:
time_series = TimeSeries("output/lambda_phage_p1_r5-3_e_tau1e-3")
mass_err01 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p1_r5-4_e_tau1e-3")
mass_err11 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p1_r5-5_e_tau1e-3")
mass_err21 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p1_r6-6_e_tau1e-3")
mass_err31 = np.abs(time_series.getMassErr())

#### Partition 2

In [68]:
time_series = TimeSeries("output/lambda_phage_p2_r5-3_e_tau1e-3")
mass_err02 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p2_r5-4_e_tau1e-3")
mass_err12 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p2_r5-5_e_tau1e-3")
mass_err22 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p2_r6-6_e_tau1e-3")
mass_err32 = np.abs(time_series.getMassErr())

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 6))

axs[0, 0].plot(time[1:], mass_err00[1:], ".-", label="$\mathcal{{P}}_0$")
axs[0, 0].plot(time[1:], mass_err01[1:], ".-", label="$\mathcal{{P}}_1$")
axs[0, 0].plot(time[1:], mass_err02[1:], ".-", label="$\mathcal{{P}}_2$")
axs[0, 0].set_title("$r = (5, 3)$")

axs[0, 1].plot(time[1:], mass_err10[1:], ".-")
axs[0, 1].plot(time[1:], mass_err11[1:], ".-")
axs[0, 1].plot(time[1:], mass_err12[1:], ".-")
axs[0, 1].set_title("$r = (5, 4)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

axs[1, 0].plot(time[1:], mass_err20[1:], ".-")
axs[1, 0].plot(time[1:], mass_err21[1:], ".-")
axs[1, 0].plot(time[1:], mass_err22[1:], ".-")
axs[1, 0].set_title("$r = (5, 5)$")

axs[1, 1].plot(time[1:], mass_err30[1:], ".-")
axs[1, 1].plot(time[1:], mass_err31[1:], ".-")
axs[1, 1].plot(time[1:], mass_err32[1:], ".-")
axs[1, 1].set_title("$r = (6, 6)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs, xlabel="$t$", xticks=[1.0, 4.0, 7.0, 10.0], ylim=[3e-8, 5e-5], yscale="log")
plt.subplots_adjust(hspace=0.5)
fig.suptitle("$|\Delta m(t)|$", fontsize=16, y=1.05)
fig.legend(*axs[0, 0].get_legend_handles_labels(), ncols=3, loc="center", bbox_to_anchor=(0.5, 0.95))
plt.tight_layout()
plt.savefig("plots/lambda_phage_mass_err_comparison_ttn.pdf");

In [ ]:
fig, ax = plt.subplots()
ax.plot(time[1:], mass_err01[1:], ".-", label="$r = (5, 3)$")
ax.plot(time[1:], mass_err11[1:], ".-", label="$r = (5, 4)$")
ax.plot(time[1:], mass_err21[1:], ".-", label="$r = (5, 5)$")
ax.plot(time[1:], mass_err31[1:], ".-", label="$r = (6, 6)$")
ax.legend()
ax.ticklabel_format(style='sci', axis='y', scilimits=(-2,2))
plt.setp(ax, xlabel="$t$", ylabel="$|\Delta m(t)|$", xticks=[1.0, 4.0, 7.0, 10.0])
plt.savefig("plots/lambda_phage_mass_err_comparison_ttn_rank.pdf");

## Wall time

### Load data

#### Partition 0

In [71]:
time_series = TimeSeries("output/lambda_phage_p0_r5-3_e_tau1e-3")
walltime00 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
walltime10 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
walltime20 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p0_r6-6_e_tau1e-3")
walltime30 = time_series.getWallTime()

#### Partition 1

In [72]:
time_series = TimeSeries("output/lambda_phage_p1_r5-3_e_tau1e-3")
walltime01 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p1_r5-4_e_tau1e-3")
walltime11 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p1_r5-5_e_tau1e-3")
walltime21 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p1_r6-6_e_tau1e-3")
walltime31 = time_series.getWallTime()

#### Partition 2

In [73]:
time_series = TimeSeries("output/lambda_phage_p2_r5-3_e_tau1e-3")
walltime02 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p2_r5-4_e_tau1e-3")
walltime12 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p2_r5-5_e_tau1e-3")
walltime22 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p2_r6-6_e_tau1e-3")
walltime32 = time_series.getWallTime()

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(6, 4.5))
labels = ["$\mathcal{{P}}_{}$".format(i) for i in range(3)]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
walltime0 = [walltime00, walltime01, walltime02]
axs[0, 0].bar(labels, walltime0, color=colors)
axs[0, 0].set_title("$r = (5, 3)$")

walltime1 = [walltime10, walltime11, walltime12]
axs[0, 1].bar(labels, walltime1, color=colors)
axs[0, 1].set_title("$r = (5, 4)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

walltime2 = [walltime20, walltime21, walltime22]
axs[1, 0].bar(labels, walltime2, color=colors)
axs[1, 0].set_title("$r = (5, 5)$")

walltime3 = [walltime30, walltime31, walltime32]
axs[1, 1].bar(labels, walltime3, color=colors)
axs[1, 1].set_title("$r = (6, 6)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.subplots_adjust(wspace=0.06, hspace=0.5)
fig.suptitle("wall time [$\mathrm{s}$]", fontsize=16)

plt.savefig("plots/lambda_phage_wall_time_comparison_ttn.pdf", bbox_inches="tight");

## Memory requirements

In [75]:
def memoryRequirementsP0(rank):
    x0 = rank[0] * 656
    x10 = rank[1] * 121
    x11 = rank[1] * 11
    q = 1 * rank[0] ** 2
    q1 = rank[0] * rank[1] ** 2
    return (x0 + x10 + x11 + q + q1) * 8 * 1e-3

def memoryRequirementsP1(rank):
    x00 = rank[1] * 656
    x01 = rank[1] * 121
    x1 = rank[0] * 11
    q = 1 * rank[0] ** 2
    q0 = rank[0] * rank[1] ** 2
    return (x00 + x01 + x1 + q + q0) * 8 * 1e-3

def memoryRequirementsP2(rank):
    x00 = rank[1] * 656
    x01 = rank[1] * 11
    x1 = rank[0] * 121
    q = 1 * rank[0] ** 2
    q0 = rank[0] * rank[1] ** 2
    return (x00 + x01 + x1 + q + q0) * 8 * 1e-3

def memoryRequirementsPfull(rank):
    x0 = rank[0] * 656
    x1 = rank[0] * 1331
    q = 1 * rank[0] ** 2
    return (x0 + x1 + q) * 8 * 1e-3

In [ ]:
rank = [5, 3]
fig, axs = plt.subplots(2, 2, figsize=(6, 4))
labels = ["$\mathcal{{P}}_{}$".format(i) for i in range(3)]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]
axs[0, 0].bar(labels, memory_req, color=colors)
axs[0, 0].set_title("$r = (5, 3)$")

rank = [5, 4]
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]
axs[0, 1].bar(labels, memory_req, color=colors)
axs[0, 1].set_title("$r = (5, 4)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

rank = [5, 5]
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]
axs[1, 0].bar(labels, memory_req, color=colors)
axs[1, 0].set_title("$r = (5, 5)$")

rank = [6, 6]
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]

axs[1, 1].bar(labels, memory_req, color=colors)
axs[1, 1].set_title("$r = (6, 6)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.subplots_adjust(hspace=0.5)
fig.suptitle("memory [$\mathrm{kB}$]", fontsize=16)

plt.savefig("plots/lambda_phage_memory_comparison_ttn.pdf");